In [ ]:
from flask import Flask, request, render_template, redirect, url_for
import mysql.connector

app = Flask(__name__)

db_config = {
    'user': 'root',
    'password': 'peter930512',
    'host': 'localhost',
    'database': 'sakila'
}

@app.route('/', methods=['GET', 'POST'])
def index():
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    if request.method == 'POST':
        # 获取表单输入的多个字段
        address = request.form['address']
        address2 = request.form['address2']
        district = request.form['district']
        city_id = request.form['city_id']
        postal_code = request.form['postal_code']
        phone = request.form['phone']
        # 如果没有提供 location，设置为 NULL
        location = request.form.get('location', None)

        # 插入到数据库
        insert_query = """
            INSERT INTO address (address, address2, district, city_id, postal_code, phone, location, last_update)
            VALUES (%s, %s, %s, %s, %s, %s, ST_GeomFromText(%s), NOW())
        """
        # 使用 None 如果 location 不存在
        cursor.execute(insert_query, (address, address2, district, city_id, postal_code, phone, 'POINT(0 0)' if not location else location))
        conn.commit()

    # 查询所有已插入的地址
    select_query = "SELECT address, address2, district, city_id, postal_code, phone FROM address"
    cursor.execute(select_query)
    addresses = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('index.html', addresses=addresses)


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Sep/2024 10:42:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 10:42:48] "POST / HTTP/1.1" 200 -
